# Practica 4 - Orientada a grafos
## Sistema de recomendaciones

Descargar los datos:
http://www.datos.economia.gob.mx/NormatividadMercantil/PROFECO_2015_vEspecial.csv
Crear una base de datos de tipo grafo con los datos diseñada para
una aplicación que permita:

- Dado un estado y un producto, buscar lugares donde pueda encontrarlo.

- Dado un estado y una tienda, verificar si tiene algun incumplimiento con un producto.

- Dado un estado y un producto, buscar alternativas sin incumplimiento de ese producto o categoría

Además, su aplicación deben ser capaz de:
- Llevar un registro de las compras y lugares que ha hecho cada usuario.

- Recomendar a un usuario una tienda en done pueda encontrar en un solo lugar, lo que compra en diferentes tiendas.

- Encontrar los estados con mayor y menor incumplimiento relativo al número de tiendas que tiene.

- Recomendar productos a un usuario según lo que otros usuarios han comprado. Es decir, Dado un usuario y un producto, obtener todos los productos de esa misma categoría que han comprado otros usuarios que también han comprado ese mismo producto.

In [ ]:
# librerías
import random
import pandas as pd

from neo4j import GraphDatabase
from tqdm import tqdm_notebook

In [ ]:
profeco_df = pd.read_csv('PROFECO_2015_vEspecial.csv', encoding='latin-1')
#profeco_df.sample(3)

# Preprocesamiento: solución al problema de ingresar datos a Neo4j
profeco_df['RAZON SOCIAL VISITADA'] = profeco_df['RAZON SOCIAL VISITADA'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()
profeco_df['INCUMPLIMIENTO'] = profeco_df['INCUMPLIMIENTO'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()
profeco_df['TIPO DE PRODUCTO'] = profeco_df['TIPO DE PRODUCTO'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()
profeco_df['DESCRIPCION DEL PRODUCTO'] = profeco_df['DESCRIPCION DEL PRODUCTO'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()
profeco_df['MUNICIPIO'] = profeco_df['MUNICIPIO'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()
profeco_df['ENTIDAD'] = profeco_df['ENTIDAD'].str.replace('"','').str.replace(r'\s+', ' ', regex=True).str.lower().str.strip()


De acuerdo al modelado de datos, se tienen 4 tipos de nodos distintos:

* Productos

* Tiendas

* Municipios

* Estados

Con esto en mente, podemos crear las relaciones para los 4 tipos de nodos distintos.

## Diseño del modelo de la aplicación:

In [ ]:
# conexión con la base de datos
data_base_connection = GraphDatabase.driver(uri="bolt://localhost:7687",
auth=("neo4j", "1234"))
session = data_base_connection.session()

# valores del DataFrame como listas de listas
profeco_lista = profeco_df.values.tolist()

# construcción de lista de intrucciones para Neo4j
instrucciones_neo4j = []

for transaccion in profeco_lista[:]:
    instruccion=f'''
    MERGE (e:Estado {{nombre_estado:"{transaccion[10]}"}})
    MERGE (tp:TipoProcuto {{tipo: "{transaccion[5]}"}})
    MERGE (t:Tienda {{razon_social:"{transaccion[7]}"}})
    MERGE (m:Municipio {{nombre_municipio:"{transaccion[9]}"}})
    MERGE (p:Producto {{nombre_producto:"{transaccion[6]}", estado:e.nombre_estado,
    municipio:m.nombre_municipio}})
    MERGE (p)-[et:en_tienda {{incumplimiento:"{transaccion[11]}"}}]->(t)
    MERGE (t)-[:en_municipio]->(m)
    MERGE (m)-[:en_estado]->(e)
    MERGE (p)-[:es_de_tipo]->(tp)
    '''.replace('\n',' ')

    instrucciones_neo4j.append(instruccion)

# ejecutar instrucciones de instrucciones_neo4j
for i in notebook.tqdm(instrucciones_neo4j):
    session.run(i)

## Consulta 1
Dado un estado y un producto, buscar lugares donde pueda encontrarlo.

In [ ]:
# code here...

def estado_producto(estado,producto):
    '''
    Regresa la tienda y municipio que tiene ese producto
    '''
    query=f'''
MATCH(m:Municipio)
MATCH(e:Estado {{nombre_estado:"{estado}"}})
MATCH(t:Tienda)
MATCH(p:Producto {{nombre_producto:"{producto}",estado:"{estado}",municipio:m.nombre_municipio}})
MATCH (p)-[:en_tienda]->(t)
MATCH (m)-[:en_estado]-> (e)
MATCH (t)-[:en_municipio]->(m)
RETURN t,m,e'''
    q_res=session.run(query)
    return q_res.data()

In [ ]:
estado='baja california'
producto='productos con informacion comercial'
estado_producto(estado,producto)

## Consulta 2
Dado un estado y una tienda, verificar si tiene algun incumplimiento con un producto.

In [ ]:
def lack_product(tienda, estado):

    query = f"""
    MATCH (m:Municipio)
    MATCH (p:Producto {{estado:"{estado}", municipio:m.nombre_municipio}})
    MATCH (e:Estado {{nombre_estado:"{estado}"}})
    MATCH (t:Tienda {{razon_social:"{tienda}"}})
    MATCH (p) -[et:en_tienda]-> (t)
    WHERE NOT et.incumplimiento = "no se detecto incumplimiento"
    MATCH (t) -[:en_municipio]-> (m)
    MATCH (m) -[:en_estado]-> (e)
    RETURN p
    """.replace('\n',' ')

    return session.run(query).data()

## Consulta 3
Dado un estado y un producto, buscar alternativas sin incumplimiento de ese producto o categoría.

In [ ]:
def buscar_alternativas(estado, producto):
    query = f'''
    MATCH (p:Producto)-[et:en_tienda]->(t:Tienda)
    MATCH (e:Estado)
    MATCH (m:Municipio)
    MATCH (t)-->(m)-->(e)
    WHERE et.incumplimiento <> "no se detecto incumplimiento"
        AND e.nombre_estado = "{estado}" AND p.nombre_producto = "{producto}"
    RETURN p,t,m,e
    '''
    alternativa = session.run(query)
    return alternativa.data()

In [ ]:
nombre_producto = 'cajacon reproductor de disco de video digital dvd mrca philips mod dvp2800'
alternativas = buscar_alternativas('mexico', nombre_producto)
print('--Alternativas de lugares donde se puede encontrar sin incumplimiento el producto--')
for i, alternativa in enumerate(alternativas):
    print(f"Alternativa {i+1}: {alternativa['m']['nombre_municipio']}, {alternativa['t']['razon_social']}")

## Funcionalidad 1 
Llevar un registro de las compras y lugares que ha hecho cada usuario.

In [ ]:
# code here...

def registrar_compra(user,curp,tienda,producto,estado,municipio,año,mes,dia):
  q_compra_usuario=f'''
  MERGE (p: Producto {{nombre_producto:"{producto}",municipio:"{municipio}",estado:"{estado}"}})
  MERGE (t: Tienda {{razon_social:"{tienda}"}})
  MERGE (m: Municipio {{nombre_municipio:"{municipio}"}})
  MERGE (p)-[:en_tienda]->(t)
  MERGE (t)-[:en_municipio]->(m)
  MERGE (u: Usuario {{CURP:"{curp}",nombre:"{user}"}})
  MERGE (u)-[cp:compro_producto {{hora:date({{year: {año}, month: {mes}, day: {dia}}})}}]->(p)
  RETURN u,p,t,m,cp.hora
  '''
  print(f'User: {user}\nProducto: {producto}\nMunicipio: {municipio}\nfecha: {dia}/{mes}/{año}\n----------------------------------------')
  return session.run(q_compra_usuario).data()



#### Hacemos registros de datos existentes con un usuario K. Reeves

In [ ]:
user='K. Reeves'
curp='420'
for i in range(5):
  rand=random.randint(0,len(profeco_df))
  tienda=prod=profeco_df['RAZON SOCIAL VISITADA'].iloc[rand]
  municipio=profeco_df['MUNICIPIO'].iloc[rand]
  estado=profeco_df['ENTIDAD'].iloc[rand]
  producto=profeco_df['DESCRIPCION DEL PRODUCTO'].iloc[rand]
  dia=random.randint(1,28)
  mes=random.randint(1,12)
  año=random.randint(2019,2021)
  registrar_compra(user,curp,tienda,producto,estado,municipio,año,mes,dia)


#### Definimos una función para recuperar el historial de búsqueda de un usuario

In [ ]:
def historial_usuario(user,curp):

  q_historial=f'''
  MATCH (u: Usuario {{CURP:"{curp}",nombre:"{user}"}})
  MATCH (u)-[cp:compro_producto]->(p:Producto)
  RETURN u,p,cp.hora
  '''
  l=session.run(q_historial).data()
  
  municipio=[]
  producto=[]
  estado=[]
  hora=[]
  for li in l:
    municipio.append(li['p']['municipio'])
    estado.append(li['p']['estado'])
    producto.append(li['p']['nombre_producto'])
    hora.append(li['cp.hora'])

  return pd.DataFrame({'user':user,'estado':estado,'municipio':municipio,'hora':hora,'producto': producto})
  
historial_usuario(user,curp)

## Funcionalidad 2 
Encontrar los estados con mayor y menor incumplimiento
relativo al número de tiendas que tiene.

In [ ]:
def incumplimiento_por_tienda(estado):

    query = f"""
    MATCH (e:Estado {{nombre_estado:"{estado}"}})
    MATCH (m:Municipio)
    MATCH (p:Producto {{estado:"{estado}", municipio:m.nombre_municipio}})
    MATCH (t:Tienda)
    MATCH (t) -[:en_municipio]-> (m)
    MATCH (m) -[:en_estado]-> (e)
    MATCH (p) -[et:en_tienda]-> (t)
    WHERE NOT et.incumplimiento = "no se detecto incumplimiento"
    RETURN p, t
    """

    return session.run(query)

def tiendas_por_estado(estado):

    query = f"""
    MATCH (e:Estado {{nombre_estado:"{estado}"}})
    MATCH (m:Municipio)
    MATCH (p:Producto {{estado:"{estado}", municipio:m.nombre_municipio}})
    MATCH (t:Tienda)
    MATCH (t) -[:en_municipio]-> (m)
    MATCH (m) -[:en_estado]-> (e)
    MATCH (p) -[et:en_tienda]-> (t)
    RETURN count(t)
    """

    return session.run(query).data()[0]['count(t)']

faltantes = {}
for estado in profeco_df.ENTIDAD.unique():

    tiendas = set()
    for dict in incumplimiento_por_tienda(f'{estado}').data():
        tiendas.add(dict['t']['razon_social'])

    faltantes[f'{estado}'] = len(tiendas) / tiendas_por_estado(f'{estado}')

# creamos un DataFrame con los resultados obtenidos
faltas_por_estado = pd.Series(faltantes).sort_values(ascending=False)

# graficamos los resultados
fig = px.bar(faltas_por_estado)
fig.update_traces(
    marker_color='rgb(158,202,225)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
    )
fig.update_layout(
    title='Porcentaje de tiendas con artículos faltantes',
    xaxis_title='Estado',
    yaxis_title='Porcentaje',
    showlegend=False,
    width=900,
    template='plotly_dark'
)
fig.write_image('Images/tiendas_faltantes.pdf')
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dd92f94b-7239-46d3-b575-e515589eae55' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>